In [ ]:
# Cluster Analysis for Customer Segmentation

# Alejandro Pineda, PhD


# Methods: KModes clustering (categorical variables), Shapley values (feature importance), Centroid analysis (profiles)


### lookup OPTICS on sklearn to measure density.

# Load Libraries (install as necessary)
import numpy as np
import pandas as pd
import sklearn
import os
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist 
#from sklearn import StandardScalar
from kmodes.kprototypes import KPrototypes
from kmodes.kmodes import KModes
# seed # (might be possible to set numpy seed globally, but not advised? Investigate further)
rando_n = 1234
# from lightgbm import LGBMClassifier
# import shap
from sklearn.model_selection import cross_val_score
from plotly import graph_objects as go
from tqdm import tqdm
import re

from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn import datasets

from prince import FAMD
import sklearn
import altair as alt 
alt.data_transformers.disable_max_rows()
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb

In [ ]:
def data_loader(file_name, col_drops):
    """
    Loads data//drops columns where necessary//provides shape and dtype info
    
    """
    original_dat = pd.read_csv(file_name, low_memory=False)
    og_shape = original_dat.shape
    
    dat_dropped = original_dat.drop(columns=col_drops, errors = 'ignore') # see if we can do try/except here
    drop_shape = dat_dropped.shape
    
    dat_types = pd.DataFrame(dat_dropped.dtypes)
    print("Header for new data:")
    print(dat_dropped.head())
    print(f"Shape of original data: {og_shape}" )
    print(f"Shape of new data: {drop_shape}")

    
    return dat_dropped, drop_shape, dat_types

In [ ]:
# Some of our column names have weird names or tags 
# (legacy data will use tags to tell use what database it came from)
# use reg-ex to pattern match and clean

# this function needs work

def pattern_find(columns, pattern=str):
    """
    Looks for columns that match a specific pattern
    """
    reg_pattern = ".*" + pattern
    display(f"Looking for the following pattern: {reg_pattern}")
    
    r = re.compile(reg_pattern)
    col_list = list(filter(r.match, columns)) 
    display(col_list[0:5])
    return col_list
    

In [ ]:
# Handling missing data and identifying categorical columns by index


def prep_dat(data_frame, thresh=None, mean_fill = False, dat_fill=None, limit=None, na_drop=None):

    """
    Fills in and/or drops missing values. 
    
    dat_fill (str): ‘backfill’, ‘bfill’, ‘pad’, ‘ffill’, None (default)
    limit (int): the maximum number of consecutive NaN allowed
    na_drop (str): Do you want rows or columns dropped? 'columns','index', or None (default) 
    thresh (int): Do you want to just require a specific # of non-NA values?
    mean_fill: Do you want missing values in numerical columns filled in with column mean? Boolean
    """
    
    data_frame2 = data_frame
    
    num_cols = data_frame2.select_dtypes(exclude=['object']).columns
    data_frame2[num_cols] = data_frame2[num_cols].astype(np.float32)
    
    # handle numerical missing nulls with the mean
    if mean_fill:
        num_features = list(num_cols)
        data_frame2[num_features]  = data_frame2[num_features].fillna(data_frame2.mean())
    
    # handle categorical missing nulls with the specified method
    if dat_fill:
        data_frame2 = data_frame2.fillna(method=dat_fill, limit = limit)
    
    if na_drop:  
        data_frame2 = data_frame2.dropna(axis=na_drop, thresh=thresh)
    
    display(f"Data loaded for this model:")
    display(data_frame.head())
    
    display("Data after pre-processing: ")
    num_features = list(data_frame2.select_dtypes(exclude=['object']).columns)
    data_frame2[num_features] = StandardScaler().fit_transform(data_frame2[num_features])
    
    
    # Identifying categorical variables based on index
    all_cols = list(data_frame2.columns)
    
    cat_col_names = list(set(all_cols) - set(num_features))
    cat_index = list()
    for col in cat_col_names:
        # find the index no
        index_loc = data_frame2.columns.get_loc(col)
        cat_index.append(index_loc)

    display(f"n_categorical features: {len(cat_index)}")
    display(f"n_numerical features: {len(num_features)}")
    
    display(f"Shape of the original data set: {data_frame.shape} // Shape of new data: {data_frame2.shape}")
    display("Pre-processed data:")
    display(data_frame2.head())
    return data_frame2
        



In [ ]:
# Columns we know we won't need
# either the columns represent duplicate information or are just useless


# lil_trash = ['IndividualID','FAME-Hip Hop']

# display("Number of trash columns: ", len(lil_trash))


# folder = 'C:/OneDrive/OneDrive - YRBrands/Documents/data/Ford_Owner/Ford Owner/'
# folder = 'C:/OneDrive/OneDrive - YRBrands/Documents/data/'
# folder = 'C:/Users/Alejandro.Pineda/Downloads/May24_Ford_Data/'
folder = "C:/Users/Alejandro.Pineda/Downloads/"

os.listdir(folder)

In [ ]:

filename1 = folder +  'data_file.csv' 

display(filename1)

dat1 = pd.read_csv(filename1, header=0)

display(dat1.head(100))
display(dat1.shape)



In [ ]:
dat1 = dat1.sample(frac=1, replace=False, random_state=rando_n)
dat1.head()

In [ ]:
list(dat1.columns)

In [ ]:
#subset_df = dat1.sample(frac=.20, replace=False, random_state=rando_n)
#subset_df.shape

In [ ]:
out_folder = "path/to/output/folder"
out_folder

In [ ]:
dat1 = dat1.dropna(axis = 'rows', how='any')
#dat3 = dat2.dropna(axis = 'columns', thresh = .75)

#dat1 = subset_df
dat1.shape

In [ ]:
dat1.head()

In [ ]:
cols = list(dat1.columns)
trash = list()

for i in cols:
    string = i.lower()
    if 'id' in string:
        print(i)
        trash.append(i)

In [ ]:
#trash.remove('DMA_ID')
#trash

In [ ]:
trash = ['cols_we_dont_need']


dat1.drop(columns = trash, inplace = True)
dat1.columns


In [ ]:
# Checking for null values (data quality is important!)

#dat = dat.dropna(axis=1)
nulls = dat1.isnull().agg('sum')
nulls

In [ ]:
#dat1.dropna(axis = 'rows', how='any', inplace=True)
#nulls = dat3.isnull().agg('sum')
#nulls

In [ ]:
dat1.shape

In [ ]:
dtype_dict = dat1.dtypes.to_dict()
dtype_dict

In [ ]:
# num_features = list(dat2.select_dtypes(exclude=['object']).columns)

# dat2.select_dtypes(exclude=['object']).columns


# dat3 is downsampled from dat2

# dat3 = dat2.sample(n=50000, random_state = rando_n)

# dat4 is set aside for output, preserving original values
# dat4 = dat3

# dat3 gets scaled
#num_features = list(dat2.select_dtypes(exclude=['object']).columns)
#dat2[num_features] = StandardScaler().fit_transform(dat2[num_features])
#dat2

In [ ]:
# dat_prepped = prep_dat(dat2, thresh=None, mean_fill = False, dat_fill='ffill', limit=25, na_drop='rows')


In [ ]:
def kmodes_trainer(data_frame, k, random_state, num_init=3):
    
    proto_dat = data_frame
        
    #### Initialize and train model on give df ####    
    
    # Leaving init and verbose as settings we don't need to worry about for now

    km = KModes(n_clusters= k, init='Cao', verbose=2, random_state=random_state, n_init = num_init)
    display(f"Training k-modes model with {k} clusters and {num_init} centroid inits.")
    
    
    clusters = km.fit_predict(proto_dat)
    proto_dat['label'] = km.predict(proto_dat)
    data_frame_lab = proto_dat
    
    return km, clusters, data_frame_lab

In [ ]:
def kproto_trainer(data_frame, k, random_state, num_init=3):
    
    proto_dat = data_frame
    
    num_features = list(proto_dat.select_dtypes(exclude=['object']).columns)
    all_cols = list(proto_dat.columns)
    
    cat_col_names = list(set(all_cols) - set(num_features))
    cat_index = list()
    for col in cat_col_names:
        # find the index no
        index_loc = proto_dat.columns.get_loc(col)
        cat_index.append(index_loc)
    
    
    #### Initialize and train model on give df ####    
    
    # Leaving init and verbose as settings we don't need to worry about for now
    kproto = KPrototypes(n_clusters=k, init='Huang', verbose=2, random_state=random_state, n_init = num_init)
    display(f"Training k-prototype model with {k} clusters and {num_init} centroid inits.")
    
    
    clusters = kproto.fit_predict(proto_dat, categorical=cat_index)
    proto_dat['label'] = kproto.predict(proto_dat, categorical = cat_index)
    data_frame_lab = proto_dat
    
    return kproto, clusters, data_frame_lab
    

In [ ]:
proto_dat = dat1.sample(frac=.15, replace=False, random_state=rando_n)
proto_dat.shape

In [ ]:

   
num_features = list(proto_dat.select_dtypes(exclude=['object']).columns)
all_cols = list(proto_dat.columns)
    
cat_col_names = list(set(all_cols) - set(num_features))
cat_index = list()

for col in cat_col_names:
    # find the index no
    index_loc = proto_dat.columns.get_loc(col)
    cat_index.append(index_loc)
    
costs = []
n_clusters = []
clusters_assigned = []

for i in tqdm(range(2, 9)):
    try:
        kproto = KPrototypes(n_clusters= i, init='Cao', verbose=2)
        clusters = kproto.fit_predict(proto_dat, categorical = cat_index)
        costs.append(kproto.cost_)
        n_clusters.append(i)
        clusters_assigned.append(clusters)
    except:
        print(f"Can't cluster with {i} clusters")
        
fig = go.Figure(data=go.Scatter(x=n_clusters, y=costs ))
fig.show()

In [ ]:
# Elbow plot with cost (will take a LONG time)
  
costs = []
n_clusters = []
clusters_assigned = []

for i in tqdm(range(2, 9)):
    try:
        km = KModes(n_clusters= i, init='Cao', verbose=2)
        clusters = km.fit_predict(proto_dat)
        costs.append(km.cost_)
        n_clusters.append(i)
        clusters_assigned.append(clusters)
    except:
        print(f"Can't cluster with {i} clusters")
        
fig = go.Figure(data=go.Scatter(x=n_clusters, y=costs ))
fig.show()


In [ ]:
# Training the model
# three clusters going forward (looking at elbow plot above)

km_model, km_cluster, lab_df = kmodes_trainer(data_frame=proto_dat, random_state=rando_n, k=3, num_init=10)

In [ ]:
km_model.cluster_centroids_

In [ ]:
km_model

In [ ]:
# make sure this parallels output file name
print(out_folder)

In [ ]:
import pickle



model_file = "demographics_query.pkl"

output_file_name = out_folder + model_file
output_file_name


pickle.dump(km_model, open(output_file_name, 'wb'))



In [ ]:

with open(output_file_name, 'rb') as file:  
    pickled_km = pickle.load(file)

pickled_km




In [ ]:
"""
Analysis: tell a story, know your audience, give them the magic***



***explain the magic in appendices and be ready to answer any technical questions, i.e., why you did something. 

"""

In [ ]:
# resample and verify that the index matches up before stapling the column back
#dat3 = dat2.sample(n=50000, random_state = rando_n)
#display(dat3.head())
#display(lab_df.head())

In [ ]:
# make sure this parallels output file name
print(filename1)

In [ ]:
#fname = out_folder + "clusters_renewal_may24.csv"
#lab_df = pd.read_csv(fname)
#lab_df

In [ ]:
#dat3['label'] = lab_df['label']
#display(dat2.head(50))
#display(lab_df)

fname = "clusters_demographics_military.csv"
out_file = out_folder + fname
display(out_file)

lab_df.to_csv(out_file, index = False)
#lab_df = pd.read_csv(out_file)
#lab_df

In [ ]:
"""
model_file = "clusters_nonluxury_midsize_pickup.csv"


output_file_name = out_folder + model_file
output_file_name

lab_df.to_csv(output_file_name, index=False)
"""


In [ ]:
# Print cluster centroids of the trained model.
print(pickled_km.cluster_centroids_)

# Print training statistics
#print(pickled_km.cost_)
#print(pickled_km.n_iter_)


#kproto_model, kproto_cluster, lab_df


In [ ]:
pickled_km.cluster_centroids_

In [ ]:
os.listdir(out_folder)

In [ ]:
out_folder

In [ ]:

"""
famd = FAMD(n_components=2, n_iter=3, check_input=True, engine='sklearn', random_state=rando_n)

famd = famd.fit(lab_df.drop('label', axis='columns'))
famd.fit(lab_df)
famd.transform(lab_df)

famd.plot(
    lab_df,
    x_component=0,
    y_component=1
)
"""




#clx = famd.plot_row_coordinates(lab_df, figsize=(15,10), color_labels = lab_df['cluster'])
#clx.get_figure() #.savefig('clusters_822.png')

In [ ]:
lab_df

In [ ]:
lgb.__version__

In [ ]:
# Grabbing F1 score to see how well groupings based on categories predicts data
# https://en.wikipedia.org/wiki/LightGBM


proto_labs = list(lab_df['label'])
#np.float = float

#Setting the objects to category 
cat_data = lab_df.drop('label', axis='columns')

for i in cat_data.select_dtypes(include='object'):
    cat_data[i] = cat_data[i].astype('category')

clf_kp = lgb.LGBMClassifier()

#cv_scores_kp = cross_val_score(clf_kp, cat_data, lab_df, scoring='f1_weighted')
#print(f'CV F1 score for K-Prototypes clusters is {np.mean(cv_scores_kp)}')

# So this method is taking a weak learner, a gradient boosted model, to see how well the data maps onto
# the cluster labels using the categories
#(https://en.wikipedia.org/wiki/Gradient_boosting#Gradient_tree_boosting)

In [ ]:
# https://shap-lrjball.readthedocs.io/en/docs_update/generated/shap.TreeExplainer.html
import shap
import numba
clf_kp.fit(cat_data, proto_labs)
explainer_kp = shap.TreeExplainer(clf_kp)
shap_values_kp = explainer_kp.shap_values(cat_data)


In [ ]:
# display(os.listdir(out_folder))
display(filename1)

plt_title = 'Impactful Features: Military Demographics Query (October 2024)'
fname = out_folder + 'shaps_demographics_military.png'
display(fname)

In [ ]:
shap.summary_plot(shap_values_kp, cat_data, color=plt.get_cmap("tab10"), show=False)
fig = plt.gcf()
fig.set_figheight(12)
fig.set_figwidth(25)
plt.title(plt_title)
plt.legend(loc='lower right')
plt.savefig(fname)

In [ ]:
#dat_prepped = prep_dat(lab_df, thresh=None, mean_fill = True, dat_fill='ffill', limit=25, na_drop='columns')
lab_df

In [ ]:
# visualizing clusters using scatterplot
from sklearn.decomposition import PCA
# HIERARCHICAL CLUSTERING
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster import hierarchy as shc
import gower
import prince
# we're using gower's distance because it can capture differences between mixed vars
# https://medium.com/analytics-vidhya/gowers-distance-899f9c4bd553

df_subsample = lab_df.sample(n=1000, random_state = rando_n)
display(df_subsample.head())

distance_matrix = gower.gower_matrix(df_subsample)
distance_matrix

"""
for c in clusters:
    grid= sns.FacetGrid(clusters, col='cluster')
    grid.map(plt.hist, c)
"""

In [ ]:
distance_matrix = pd.DataFrame(distance_matrix)

In [ ]:
labs = df_subsample.label
display(filename1)

In [ ]:
# spot check data name
plt_file = out_folder + 'demo_military_scatter.png'
display(plt_file)
plt_title = 'Demographics Query, Military (October 24)'

In [ ]:
mca = prince.MCA(
    n_components=2,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
mca = mca.fit(distance_matrix)

In [ ]:
#mca.plot(
#    distance_matrix,
#    x_component=0,
#    y_component=1)

In [ ]:
distance_matrix

In [ ]:
mca = prince.MCA(2)
mca.fit(distance_matrix)
X_PCA = mca.transform(distance_matrix)
X_PCA.shape

In [ ]:
X_PCA.loc[:,1]

In [ ]:
X_PCA

In [ ]:
def rand_jitter(arr):
    stdev = .01 * (max(arr) - min(arr))
    return arr + np.random.randn(len(arr)) * stdev

#x = rand_jitter(X_PCA.loc[:,0])
#y = rand_jitter(X_PCA.loc[:,1])

In [ ]:
labs = labs.reset_index()

In [ ]:
labs

In [ ]:
# PCA on the distance matrix and then scatterplot to visualize
# dope code for future reference: https://www.kaggle.com/code/sabanasimbutt/clustering-visualization-of-clusters-using-pca



x, y = X_PCA.loc[:,0], X_PCA.loc[:,1]

colors = {0: 'red',
          1: 'blue',
          2: 'green'}

names = {0: 'Cluster 0', 
         1: 'Cluster 1', 
         2: 'Cluster 2'}

df = pd.DataFrame({'x': x, 'y':y, 'label':labs['label']}) 
groups = df.groupby('label')

fig, ax = plt.subplots(figsize=(20, 13)) 

for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=5,
            color=colors[name],label=names[name], mec='none')
    ax.set_aspect('auto')
    ax.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off')
    ax.tick_params(axis= 'y',which='both',left='off',top='off',labelleft='off')
    
ax.legend()
ax.set_title(plt_title)
plt.savefig(plt_file)
plt.show()

"""
ax = plt.scatter(x, y, c=labs)
plt.title('Clusters: Adobe MI')
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()
"""

In [ ]:
lab_df.columns

In [ ]:
d = dict(zip(list(lab_df.columns), list(pickled_km.cluster_centroids_[0])))
d

In [ ]:
cols = lab_df.columns
c0 = list(pickled_km.cluster_centroids_[0])
c1 = list(pickled_km.cluster_centroids_[1])
c2 = list(pickled_km.cluster_centroids_[2])
#c3 = list(pickled_km.cluster_centroids_[3])
#c4 = list(pickled_km.cluster_centroids_[4])

#d = dict((z[0], list(z[1:])) for z in zip(cols, c0, c1, c2, c3))

In [ ]:
cols

In [ ]:
c0

In [ ]:
cols = cols.drop('label')
cols

In [ ]:
centroid_df = pd.DataFrame([c0,c1,c2], index = [0,1,2], columns=cols)
#centroid_df[1] = km_model.cluster_centroids_[1]
centroid_df

fname = out_folder + 'demo_military_centroids_oct24.csv'
fname
centroid_df.to_csv(fname)

In [ ]:
fname